In [1]:
import pandas as pd
from datetime import datetime
from geopy.distance import geodesic
data = pd.read_csv('unique_mls_db_GIT.csv')
cpi=pd.read_csv('CPI.csv')

In [2]:
#coordinates of major go train stations in mississauga and nearby cities: Lisgar, Meadowvale, Streesville, _
#Erindale, Cocksville, Dixie, Kipling, Clarkson, Port Credit, and Long Branch

go_station_msga=[[43.5905809,-79.788362],[43.5976,-79.75369999999998],
                 [43.5762,-79.70910000000003],[43.5670973,-79.6689102],
                 [43.5819,-79.62369999999998],[43.607605,-79.5776749],
                 [43.6357039,-79.5372283],[43.51251816552179,-79.63435649871826],
                 [43.5555,-79.5874],[43.59133291164543,-79.54609036445617]]

In [3]:
#add soldyear and soldmonth columns

data_msga=data[data['city']=='Mississauga']
data_msga.loc[:,['solddate']] = pd.to_datetime(data_msga['solddate'])
data_msga.insert(len(data_msga.columns),column='soldyear',value=pd.DatetimeIndex(data_msga['solddate']).year)
data_msga.insert(len(data_msga.columns),column='soldmonth',value=pd.DatetimeIndex(data_msga['solddate']).month)

#merge with CPI file and convert nominal sold prices to real sold prices with base case = 2015-07-31 

cpi_targetrange=cpi[(pd.to_datetime(cpi['Date'])>=datetime(2015,7,1)) & (pd.to_datetime(cpi['Date'])<=datetime(2017,6,30))]
cpi_targetrange.insert(len(cpi_targetrange.columns),column='soldyear',
                       value=pd.DatetimeIndex(cpi_targetrange['Date']).year)
cpi_targetrange.insert(len(cpi_targetrange.columns),column='soldmonth',
                       value=pd.DatetimeIndex(cpi_targetrange['Date']).month)
data_msga=pd.merge(cpi_targetrange, data_msga, on=['soldyear','soldmonth'], how='outer')
cpibase=float(cpi_targetrange['PX_LAST'][pd.to_datetime(cpi_targetrange['Date'])==datetime(2015,7,31)])
data_msga.insert(len(data_msga.columns),column='cpibase',value=cpibase)
data_msga.insert(len(data_msga.columns),column='realsoldprice',value=
                data_msga['soldprice']*data_msga['cpibase']/data_msga['PX_LAST'])


c:\users\zhaojoseph\appdata\local\programs\python\python37-32\lib\site-packages\pandas\core\indexing.py:543: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[item] = s


In [4]:
#take a house's coordinates and calculate minimum distance of a house to GO stations 

data_msga_coordinate = data_msga[['latitude','longitude']].values
mindist_commu = []
for i in range(len(data_msga_coordinate)):
    temp = []
    for j in range(len(go_station_msga)):
        temp.append(geodesic(go_station_msga[j], data_msga_coordinate[i]).meters)
    mindist_commu.append(min(temp))

In [5]:
#add the minimum distance to the dataframe

data_msga.insert(len(data_msga.columns),column='mindist_commu',value=mindist_commu)

In [6]:
data_msga

,Date,PX_LAST,soldyear,soldmonth,mlsno,stno,stname,city,area,soldprice,...,bdrm,wshrm,latitude,longitude,house_condo,aptno,address,cpibase,realsoldprice,mindist_commu
0,2017-06-30,130.4,2017,6,W3854388,2500,Hargood,Mississauga,Streetsville,1050000,...,4,3,43.576264,-79.729552,house,NaN,"2500 Hargood Pl, Mississauga",127.3,1.025038e+06,1652.022445
1,2017-06-30,130.4,2017,6,W3817888,551,Cavell,Mississauga,Cooksville,668000,...,3,2,43.563612,-79.622475,house,NaN,"551 Cavell Dr, Mississauga",127.3,6.521196e+05,2034.263710
2,2017-06-30,130.4,2017,6,W3850685,3582,Birchmeadow,Mississauga,Applewood,670000,...,3,2,43.612874,-79.607933,house,NaN,"3582 Birchmeadow Cres, Mississauga",127.3,6.540721e+05,2511.870925
3,2017-06-30,130.4,2017,6,W3848786,4186,Uxbridge,Mississauga,Rathwood,804500,...,3,3,43.616948,-79.613434,house,NaN,"4186 Uxbridge Lane, Mississauga",127.3,7.853746e+05,3067.710156
4,2017-06-30,130.4,2017,6,W3835920,4597,Centretown,Mississauga,Hurontario,889500,...,4,4,43.595464,-79.664838,house,NaN,"4597 Centretown Way, Mississauga",127.3,8.683539e+05,3168.724671
5,2017-06-30,130.4,2017,6,W3831279,308,Wendron,Mississauga,Hurontario,905000,...,4,4,43.605000,-79.667619,house,NaN,"308 Wendron Cres, Mississauga",127.3,8.834854e+05,4212.472992
6,2017-06-30,130.4,2017,6,W3852453,2421,Yorktown,Mississauga,Central Erin Mills,1060000,...,4,3,43.573708,-79.716757,house,NaN,"2421 Yorktown Circ, Mississauga",127.3,1.034801e+06,677.664885
7,2017-06-30,130.4,2017,6,W3850663,5208,Forestwalk,Mississauga,Hurontario,1055000,...,4,4,43.616094,-79.650311,house,NaN,"5208 Forestwalk Circ, Mississauga",127.3,1.029919e+06,4364.622170
8,2017-06-30,130.4,2017,6,W3849806,490,Avonwood,Mississauga,Mineola,1080000,...,2,2,43.569918,-79.576084,house,NaN,"490 Avonwood Dr, Mississauga",127.3,1.054325e+06,1844.445489
9,2017-06-30,130.4,2017,6,W3786174,4611,Badminton,Mississauga,Central Erin Mills,1460000,...,4,4,43.565605,-79.694308,house,NaN,"4611 Badminton Dr, Mississauga",127.3,1.425291e+06,1677.392590
